In [ ]:
import pandas as pd

calgary_weather_data_one_path = "./datasets/calgary_weather_data_20180101_20220101.csv"
calgary_weather_data_two_path = "./datasets/calgary_weather_data_20220101_20250101.csv"

cw1_df = pd.read_csv(calgary_weather_data_one_path, encoding="latin-1")
cw2_df = pd.read_csv(calgary_weather_data_two_path, encoding="latin-1")

cw1_df = cw1_df[["Station Name", "Date (Local Standard Time)", "Air Temp. Avg. (°C)"]]
cw2_df = cw2_df[["Station Name", "Date (Local Standard Time)", "Air Temp. Avg. (°C)"]]

cw_df = pd.concat([cw1_df, cw2_df],  ignore_index=True)
display(cw_df.head())


: 

In [ ]:
community_crime_statistics_calgary_path = "./datasets/community_crime_statistics_calgary.csv"

ccsc_df = pd.read_csv(community_crime_statistics_calgary_path)


display (ccsc_df)

In [ ]:
communities_by_ward_calgary_path = "./datasets/communities_by_ward_calgary.csv"
coms_ward_df = pd.read_csv(communities_by_ward_calgary_path)

display(coms_ward_df)


: 